In [9]:
import numpy as np
import os
import sys
import pandas as pd
import csv
import glob
import shutil
import seaborn as sns
import matplotlib.pyplot as plt





def drn_in_reaction(df1, df2):
    #obtaining dRn levels of actual target in reaction from qPCR file with pseudo targets
    dRn_target_in_reaction = df2.merge(df1, how='left', on=['Target', 'Well Position'])
    
    #omitting unnecessary data, renaming certain colummns to better differentiate later on when calculating crosstalk, and calculating mean values 
    dRn_target_in_reaction = dRn_target_in_reaction.drop(['Rn', 'Omit',  'Well', 'Sample'], axis =1)
    dRn_target_in_reaction = dRn_target_in_reaction.rename(columns = {'Well Position': 'Well Position', 'Target': 'Target in Reaction', 'Cycle Number': 'Cycle Number', 'dRn': 'dRnRight'})
    dRn_target_in_reaction = dRn_target_in_reaction.set_index(['Well Position', 'Target in Reaction', 'Cycle Number'])
    dRn_target_in_reaction = dRn_target_in_reaction.groupby(['Target in Reaction', 'Cycle Number']).mean()
    
    return dRn_target_in_reaction


def pseudo_target_drn(df1, df2):
    df = df1.drop(['Rn', 'Omit',  'Well'], axis =1)
    df_qpcr_data = df
    df_in_reaction = df2.rename(columns = {'Well Position': 'Well Position','Target': 'Target in Reaction'}) # df in reaction is the plate layout
    df_merge_pseudotarget = df_in_reaction.merge(df_qpcr_data, how = 'left')  
    df_merge_pseudotarget = df_merge_pseudotarget.drop(['Sample'], axis = 1)
    # set index to calculate the mean values for pseduo target dRn mean values based on target in reaction and cycle number  
    df_merge_pseudotarget = df_merge_pseudotarget.set_index(['Target in Reaction', 'Target', 'Cycle Number'])
    df_merge_pseudotarget = df_merge_pseudotarget.groupby(['Target in Reaction', 'Target', 'Cycle Number']).mean()
    
    return df_merge_pseudotarget


# qPCR data files path 
path = r'/Users/miguelestrada/Desktop/qPCR_Projects/MultiplexCombinations/Data/CrossTalkData'

folder = os.path.join(path, 'Crosstalk Calculations')

#read all the excel files
filenames = glob.glob(path + "/*.xlsx")

# Creates cross-talk folder to input all qPCR cross-talk files in this folder
try:
    os.makedirs(folder)
except FileExistsError:
    print("CrossTalk Folder Already Exist")
    

#plate layout excel file
plate_layout = (r'/Users/miguelestrada/Desktop/qPCR_Projects/MultiplexCombinations/Plan/SOP_MultiplexCombinations.xlsx')

for file in filenames:
    excel_name = os.path.basename(file)
    crosstalk_string = '_crosstalk.csv'
    file_name = excel_name.rsplit('.', 1)[0]
    multiplex_combination = file_name.rsplit('_',-1)[-1]  # based on the last description of the file 
    file_name_crosstalk = (file_name + crosstalk_string)

    path_to_save = os.path.join(folder, file_name_crosstalk)
    df1 = pd.read_excel(file, 'Amplification Data', header = 23)  # cross-talk qPCR amplification data
    df2 = pd.read_excel(plate_layout,'Well Position')  # target in reaction 
    
    #Calculating Target in reaction dRn mean values based on plate layout  
    dRn_target_in_reaction = drn_in_reaction(df1, df2)
    
    # Calculating Pseudo target dRn mean values based on cycle numbers and target in reaction 
    df_merge_pseudotarget = pseudo_target_drn(df1, df2)
    
    #resetting index to merge target in reaction with pseudo target dRn values 
    df_merge_pseudotarget = df_merge_pseudotarget.reset_index()
    dRn_target_in_reaction = dRn_target_in_reaction.reset_index()
    
    #merging dataframes with dRn mean values for target in reaction and qPCR pseduo target 
    newdf = pd.merge(df_merge_pseudotarget, dRn_target_in_reaction, how='left', on=['Target in Reaction', 'Cycle Number'])
    
     #calculating cross-talk from dRn mean values 
    newdf['CrossTalk %'] = 100 *( newdf['dRn']/ newdf['dRnRight'])
    newdf = newdf.rename(columns = {'dRn': 'Pseudo Target dRn','dRnRight': 'Target in Reaction dRn'})
    #labeling combination 
    newdf['Multiplex Combination'] = multiplex_combination
    
     
    #saving data file 
    newdf.to_csv(path_to_save)
    

#Concateantes all files into one for JMP manipulation purposes 
os.chdir(folder)
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_crosstalk.csv", index=False, encoding='utf-8-sig')


CrossTalk Folder Already Exist


/Users/miguelestrada/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
